<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Spinning Effective One-Body Constant Coefficients

## Author: Tyler Knowles

## This module documents the reduced spinning effective one-body constant coefficients: terms calibrated to numerical relativity simulations.


**Notebook Status:** <font color='red'><b> In progress </b></font>

**Validation Notes:** This module is under active development -- do ***not*** use the resulting code for scientific applications.  In the future, this module will be validated against the LALSuite [SEOBNRv3/SEOBNRv3_opt code]( https://git.ligo.org/lscsoft/lalsuite) that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.


## Introduction
### The Physical System of Interest

Consider two compact objects (e.g. black holes or neutron stars) with masses $m_{1}$, $m_{2}$ (in solar masses) and spin angular momenta ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  Certain terms of this $H_{\rm real}$ are calibrated to numerical relativity simulations, and we document those terms here.

The fits to numerical relativity rely on the following physical parameters.
1. Let $m_{1}$, $m_{2}$ be the masses of the compact objects in units of solar mass.  We then define the symmetric mass ratio $\eta$ of the system to be (see discussion preceeding [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.1))
    \begin{equation*}
        \eta = \frac{ m_{1} m_{2} }{ \left( m_{1} + m_{2} \right)^{2} }.
    \end{equation*}
1. Let ${\bf S}_{1}$, ${\bf S}_{2}$ be the spins of the compact objects in units of FIXME.  Combining [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.64), and (5.67) we have that the spin of the deformed Kerr background is
    \begin{equation*}
        {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
    \end{equation*}
From [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9), we then define the parameter $a$ which appears in metric potenials for the Kerr spacetime as
    \begin{equation*}
        a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M^{2} }
    \end{equation*}
where $M = m_{1} + m_{2}$.

We also use the Euler–Mascheroni constant $\gamma$, [hard-coded in LALSuite](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html) with the following value:
    \begin{equation*}
        \gamma = 0.577215664901532860606512090082402431
    \end{equation*}

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant, and
1. $m_{1} \ge m_{2}$.

### Citations
Throughout this module, we refer to
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006.

LALSuite line numbers are taken from Git commit bba40f2 (see [LALSuite's GitLab page](https://git.ligo.org/lscsoft/lalsuite)).

In [2]:
# Constants of fit to numerical relativity for the spinning effective one-body formulation

# Import necessary NumPy, SymPy, and SEOBNR modules
import numpy as np

# Compute fits to numerical relativity
def compute_const_coeffs(eta, EMgamma, a):

    # Define frequently-used constants
    asq = a*a
    pisq = np.pi*np.pi

    # Define constants that determine the fitting parameter K
    # See the discussion in https://arxiv.org/pdf/1311.2544.pdf between Equations (3) and (4)
    K0 = 1.712
    K1 = -1.803949138004582
    K2 = -39.77229225266885
    K3 = 103.16588921239249

    # Compute the fitting parameter K
    # See https://arxiv.org/abs/0912.3517 Equation (5.67) and the discussion following Equation 6.9
    # as well as https://arxiv.org/pdf/1311.2544.pdf
    K = K0 + K1*eta + K2*eta*eta + K3*eta*eta*eta

    # Define more frequently-used constants
    EtaKm1 = eta*K - 1.
    EtaKm1sq = EtaKm1*EtaKm1

    # Compute the Post-Newtonian coefficients
    # See https://arxiv.org/abs/0912.3517 Equations (5.77) to (5.81) and
    # https://arxiv.org/pdf/1311.2544.pdf Equation (2)
    Delta0 = K*(EtaKm1 - 1.)
    Delta1 = -2.*(Delta0 + K)*EtaKm1

    Delta1sq = Delta1*Delta1
    Delta1cu = Delta1*Delta1sq
    Delta1ft = Delta1cu*Delta1

    Delta2 = 0.5*Delta1*(Delta1 - 4.*EtaKm1) - asq*EtaKm1sq*Delta0

    Delta2sq = Delta2*Delta2

    Delta3 = -Delta1cu/3. + Delta1*Delta2 + Delta1sq*EtaKm1 - 2.*(Delta2 - EtaKm1)*EtaKm1 - asq*Delta1*EtaKm1sq
    Delta4 = 1./12.*(6*asq*(Delta1sq - 2*Delta2)*EtaKm1sq + 3*Delta1ft - 8*EtaKm1*Delta1cu - 12*Delta2*Delta1sq
                     + 12*(2*EtaKm1*Delta2 + Delta3)*Delta1 + 12*(94./3. - 41./32.*pisq)*EtaKm1sq
                     + 6*(Delta2*Delta2 - 4*Delta3*EtaKm1))
    Delta5 = EtaKm1sq*(-4237./60. + 128./5.*EMgamma + 2275./512.*pisq - asq*(Delta1cu - 3.*Delta1*Delta2 + 3.*Delta3)/3.
                     - (Delta1ft*Delta1 - 5.*Delta1cu*Delta2 + 5.*Delta1*Delta2sq + 5.*Delta1sq*Delta3
                        - 5.*Delta2*Delta3 - 5.*Delta1*Delta4)/(5.*EtaKm1sq) + (Delta1ft - 4.*Delta1sq*Delta2
                        + 2.*Delta2sq + 4.*Delta1*Delta3 - 4.*Delta4)/(2*EtaKm1) + (256./5.)*np.log(2))
    Delta5l = (64./5.)*EtaKm1sq

    #Add comment here
    dSO = -74.71 - 156.*eta + 627.5*eta*eta
    dSS = 8.127 - 154.2*eta + 830.8*eta*eta

    return K, Delta0, Delta1, Delta2, Delta3, Delta4, Delta5, Delta5l, dSO, dSS